# Pure Python Sim
The simulation of the movement of **N bodies** in relation to each other.\
The basic simulation algorithm involves a number of iterations.\
*A description and discussion of the algorithm is found in the recent article Vladimirov, Andrey and Vadim Karpusenko (2013): "Test-Driving Intel Xeon-Phi Coprocessors with a Basic N-Body Simulation." White Paper, Colfax International.*

In [8]:
import random
import time
from numpy import sqrt

nParticles = 1000
particles = []
for i in range(nParticles):
    particles.append([random.gauss(0.0, 1.0) for j in range(3)])

particlev = [[0, 0, 0] for _ in particles]
particles[:3]

[[-0.8090111126214135, 1.6136007087233417, 0.793650447437479],
 [1.7873278271315536, -1.340990783426187, 1.0206981445212107],
 [-0.9405084351407464, 0.47910013923074113, -1.4925151448676905]]

In [39]:
def nbody(particle, particlev):  # lists as input
    t0 = time.time(); nSteps = 5; dt = 0.01
    for step in range(1, nSteps + 1):
        for i in range(nParticles):
            Fx, Fy, Fz = 0.0, 0.0, 0.0
            for j in range(nParticles):
                if j != i:
                    dx = particle[j][0] - particle[i][0]
                    dy = particle[j][1] - particle[i][1]
                    dz = particle[j][2] - particle[i][2]
                    drSquared = dx * dx + dy * dy + dz * dz 
                    drPowerN32 = 1.0 / (drSquared + sqrt(drSquared)) # 1 / distance 
                    Fx += dx * drPowerN32
                    Fy += dy * drPowerN32
                    Fz += dz * drPowerN32
                particlev[i][0] += dt * Fx
                particlev[i][1] += dt * Fy
                particlev[i][2] += dt * Fz
        for i in range(nParticles):
            particle[i][0] += particlev[i][0] * dt
            particle[i][1] += particlev[i][1] * dt
            particle[i][2] += particlev[i][2] * dt
    return time.time() - t0

In [40]:
%time ti_py = nbody(particles, particlev)

CPU times: total: 14.4 s
Wall time: 16.5 s


# NumPy 
vectorization of operation

In [13]:
import numpy as np

In [15]:
particles = np.random.standard_normal((nParticles, 3))
particlev = np.zeros_like(particles)

In [41]:
def nbody_np(particle, particlev):  # NumPy arrays as input
    t0 = time.time(); nSteps = 5; dt = 0.01
    for step in range(1, nSteps + 1, 1):
        Fp = np.zeros((nParticles, 3))
        for i in range(nParticles):
            dp = particle - particle[i]
            drSquared = np.sum(dp ** 2, axis=1)
            h = drSquared + np.sqrt(drSquared)
            # drPowerN32 = np.where(h > 0., 1. / h, 1E-10)
            drPowerN32 = 1. / np.maximum(h, 1E-10)
            Fp += -(dp.T * drPowerN32).T
            particlev += dt * Fp
        particle += particlev * dt
    return time.time() - t0

nbody_np(particles, particlev)


0.23354125022888184

In [42]:
%time ti_np = nbody_np(particles, particlev)

CPU times: total: 156 ms
Wall time: 227 ms


# Numba JIT

In [43]:
from numba import jit, objmode

@jit(nopython=True)
def nbody_nb(particle, particlev):  # NumPy arrays as input
    with objmode(t0='f8'):
        t0 = time.time()
    nSteps = 5; dt = 0.01
    for step in range(1, nSteps + 1, 1):
        for i in range(nParticles):
            Fx = 0.0; Fy = 0.0; Fz = 0.0
            for j in range(nParticles):
                if j != i:
                    dx = particle[j,0] - particle[i,0]
                    dy = particle[j,1] - particle[i,1]
                    dz = particle[j,2] - particle[i,2]
                    drSquared = dx * dx + dy * dy + dz * dz
                    drPowerN32 = 1.0 / (drSquared + sqrt(drSquared))
                    Fx += dx * drPowerN32
                    Fy += dy * drPowerN32
                    Fz += dz * drPowerN32
                particlev[i, 0] += dt * Fx
                particlev[i, 1] += dt * Fy
                particlev[i, 2] += dt * Fz
        for i in range(nParticles):
            particle[i,0] += particlev[i,0] * dt
            particle[i,1] += particlev[i,1] * dt
            particle[i,2] += particlev[i,2] * dt
    with objmode(t1 = 'f8'):
        t1 = time.time() - t0
    return t1

firstrun = nbody_nb(particles, particlev)


In [44]:
%time ti_nb = nbody_nb(particles, particlev)

CPU times: total: 15.6 ms
Wall time: 23 ms


# Optimized Numba

In [45]:
import numba

@numba.njit('(float64[:,:], float64[:,:])', cache=True, fastmath=True, parallel=True)
def nbody_nb(particle, particlev):  # NumPy arrays as input
    with objmode(t0='f8'):
        t0 = time.time()
    nSteps = 5; dt = 0.01
    for step in range(1, nSteps + 1, 1):
        for i in numba.prange(nParticles):
            Fx = 0.0; Fy = 0.0; Fz = 0.0
            for j in numba.prange(nParticles):
                if j != i:
                    dx = particle[j,0] - particle[i,0]
                    dy = particle[j,1] - particle[i,1]
                    dz = particle[j,2] - particle[i,2]
                    drSquared = dx * dx + dy * dy + dz * dz
                    drPowerN32 = 1.0 / (drSquared + sqrt(drSquared))
                    Fx += dx * drPowerN32
                    Fy += dy * drPowerN32
                    Fz += dz * drPowerN32
                particlev[i, 0] += dt * Fx
                particlev[i, 1] += dt * Fy
                particlev[i, 2] += dt * Fz
        for i in numba.prange(nParticles):
            particle[i,0] += particlev[i,0] * dt
            particle[i,1] += particlev[i,1] * dt
            particle[i,2] += particlev[i,2] * dt
    with objmode(t1='f8'):
        t1 = time.time() - t0
    return t1
nbody_nb(particles, particlev)


0.005051136016845703

In [46]:
%time ti_nb = nbody_nb(particles, particlev)

CPU times: total: 15.6 ms
Wall time: 4 ms


In [53]:
print(f"Numba speed up from numpy vectorizated operation by {ti_np / ti_nb:.0f} times")  # speed-up vs. NumPy function

Numba speed up from numpy vectorizated operation by 57 times


In [55]:
print(f"Numba speed up pure python by {ti_py / ti_nb:.0f} times")  # speed-up vs. pure Python function


Numba speed up pure python by 4126 times
